Autoencoders implementation

- can be used for Denoising Autoencoders

In [ ]:
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import numpy as np

encoding=32
inputimg=Input(shape=(784,))#shape of the input
encoded=Dense(encoding,activation='relu')(inputimg)#create layer then second bracket join the layer with a defined layer

decoded=Dense(784,activation='sigmoid')(encoded)#sigmoid used as values are scaled between 0 to 1 by dividing 255

autoencoder=Model(inputimg,decoded)#the whole model
encoder=Model(inputimg,encoded)#the encoder part

encodedinput=Input(shape=(encoding,))
decodedlayer=autoencoder.layers[-1]#last layer of the autoencoder model which is (784,)
decoder=Model(encodedinput,decodedlayer(encodedinput))#decodedlayer(encodedinput)joins the 2 layers together so essentially its the decoded part of the model

#separated encoder used for feature extraction and decoder can be used for data generation
autoencoder.compile(optimizer='adadelta',loss='binary_crossentropy')
#training
(xtrain,_),(xtest,_)=mnist.load_data()

#scaling the value betweemn 0 and 1
xtrain=xtrain.astype('float32')/255
xtest=xtest.astype('float32')/255

xtrain=xtrain.reshape(len(xtrain),np.prod(xtrain.shape[1:]))#data in shape(X,y,y) reshaped into (X,Y*Y)
xtest=xtest.reshape(len(xtest),np.prod(xtest.shape[1:]))#similar to above

autoencoder.fit(xtrain,xtrain,epochs=50,batch_size=256,validation_data=(xtest,xtest))

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.6931 - val_loss: 0.6930
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6930 - val_loss: 0.6929
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6928 - val_loss: 0.6927
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6927 - val_loss: 0.6926
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6925 - val_loss: 0.6924
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6924 - val_loss: 0.6923
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6923 - val_loss: 0.6922
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6921 - val_loss: 0.6920
Epoch 9/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6920 - val_loss: 0.6919
Epoch 10/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6918 - val_loss: 0.6917
Epoch 11/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6917 - val_loss: 0.6916
Epoch 12/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step